# Capstone

In [1]:
#  and len(word.split())>1

## Noun-Phrase and PMI Analysis

#### Installing required libraries

In [145]:
!pip --quiet install nltk
import nltk
nltk.download("vader_lexicon")
nltk.download("stopwords")

#for candidate key phrase code
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download("wordnet")
nltk.download("brown")

from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.stem import *
from nltk.tag import PerceptronTagger
from nltk.data import find
import pandas
import numpy as np
import math


import json
from collections import Counter

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/notebook/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/notebook/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/notebook/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/notebook/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/notebook/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to /home/notebook/nltk_data...
[nltk_data]   Package brown is already up-to-date!


#### Noun Phrase - Implementation 1
Source: http://bdewilde.github.io/blog/2014/09/23/intro-to-automatic-keyphrase-extraction/

In [3]:
def extract_candidate_chunks(text, grammar=r'KT: {(<JJ>* <NN.*>+ <IN>)? <JJ>* <NN.*>+}'):
    import itertools, nltk, string
    
    # exclude candidates that are stop words or entirely punctuation
    punct = set(string.punctuation)
    stop_words = set(nltk.corpus.stopwords.words('english'))
    # tokenize, POS-tag, and chunk using regular expressions
    chunker = nltk.chunk.regexp.RegexpParser(grammar)
    tagged_sents = nltk.pos_tag_sents(nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(text))
    all_chunks = list(itertools.chain.from_iterable(nltk.chunk.tree2conlltags(chunker.parse(tagged_sent))
                                                    for tagged_sent in tagged_sents))
    # join constituent chunk words into a single chunked phrase
    candidates = [' '.join(word for word, pos, chunk in group).lower()
                  #python 2 to 3 fix for the following line : http://stackoverflow.com/questions/36238591/python-2-x-sublist-parameter-equivalent-in-3-x
                  for key, group in itertools.groupby(all_chunks, lambda word__pos__chunk: word__pos__chunk[2] != 'O') if key]

    return [cand for cand in candidates
            if cand not in stop_words and not all(char in punct for char in cand)]

#### Read JSON Data Into Pandas DF

In [4]:
file_in = '/resources/data/shopify_lower.json'

In [5]:

#following code courtesy of: https://www.reddit.com/r/MachineLearning/comments/33eglq/python_help_jsoncsv_pandas/
with open(file_in, 'r') as f:
    data = f.readlines()

# remove the trailing "\n" from each line
data = map(lambda x: x.rstrip(), data)

# each element of 'data' is an individual JSON object.
# i want to convert it into an *array* of JSON objects
# which, in and of itself, is one large JSON object
# basically... add square brackets to the beginning
# and end, and have all the individual business JSON objects
# separated by a comma
data_json_str = "[" + ','.join(data) + "]"

# now, load it into pandas
dataDf = pandas.read_json(data_json_str)

In [6]:
#to see all headings in data
#for column in dataDf.columns:
    #print(column)

In [7]:
#Printing sample of DF
#dataDf.head()

In [8]:
#How much Data? (Rows,Columns)
dataDf.shape

(1106, 22)

In [9]:
#do analysis on a copy of the orginal text
dataDf['bodyCopy'] =dataDf['body']

#### Filtering

In [105]:
import re
URL_REGEX_1 = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
SHOPIFY_REGEX = r"[S|s]hopify"
HIRING_REGEX = r"[H|h]iring"
POPULARCOMMENT_REGEX = r"\**Most Popular Comments\**"
HISTORY_REGEX = r"[R|r]ecent [S|s]ubmission [H|h]istory "

In [106]:
#list to store ids of comments to keep 
commentKeep =[]

for cid,comment in enumerate(dataDf['bodyCopy']):
    #print(cid)
    #print(dataDf.loc[dataDf['body']==comment].index.values[0])
    if (len(re.findall(URL_REGEX_1,comment))>0):
        dataDf['bodyCopy'][cid] = re.sub(URL_REGEX_1,'WEBSITE_FILLER',comment)
        #print(dataDf['bodyCopy'][cid])
    #only keep comments with links if they mention shopify w/o it occuring in link    
    if (len(re.findall(SHOPIFY_REGEX,comment))>0):
            #remove any post with 'Hiring' or 'Recent Submission History'
            if(len(re.findall(HIRING_REGEX,comment))==0 and  len(re.findall(HISTORY_REGEX,comment))==0 and len(re.findall(POPULARCOMMENT_REGEX,comment))==0):
                commentKeep.append(cid)
                
        
print("****Done****")


****Done****


In [108]:
#how many comments left?
print("After Filtering:",len(commentKeep),"comments have been kept from:",dataDf.shape[0])

After Filtering: 470 comments have been kept from: 1106


In [109]:
#assign subset of comments to keep to new df

commentKeepDf=dataDf.loc[commentKeep]

In [110]:
#Check Size
commentKeepDf.shape

(470, 23)

In [111]:
#commentKeepDf.columns

#### Looking at noun-phrases per comment

In [112]:
#for count,comment in enumerate(commentKeepDf['bodyCopy']):
    #print("Comment: " + comment)
    #print("\n Noun-phrases/Candidate Key Phrases:")
    #print(extract_candidate_chunks(comment) ,"\n\n")
    #print("----------------------------------------------------")

#### Building different counters to analyze top k frequent words in whole collection

In [147]:
#List of all 'english' stop words; stop words are common words
stop = set(stopwords.words('english'))
stop.add('WEBSITE_FILLER')

In [114]:
# download required resources
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /home/notebook/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [115]:
from nltk.stem import *
wnLemm = WordNetLemmatizer()
sbStem = SnowballStemmer("english")
#wnLemm.lemmatize(word)

In [116]:
#Count the frequency of words
from collections import Counter
#https://pymotw.com/2/collections/counter.html
# counter is a container that keeps track of how many times equivalent values are added
import re
#A regular expression (or RE) specifies a set of strings that matches it
#A formal language for specifying strings
counter_basic = Counter()
counter_stem = Counter()
counter_lemm = Counter()
for rnumb,comment in enumerate(commentKeepDf['bodyCopy']):
        #add the word in its lowercase form to the counter Container if it is not a stop word and is greather than 2 characters
        counter_basic.update([(word.lower()) for word in extract_candidate_chunks(comment)  if word.lower() not in stop and len(word) > 2 and len(word.split())>1])
        counter_stem.update([sbStem.stem(word.lower()) for word in extract_candidate_chunks(comment)  if word.lower() not in stop and len(word) > 2 and len(word.split())>1])
        counter_lemm.update([wnLemm.lemmatize(word.lower()) for word in extract_candidate_chunks(comment)  if word.lower() not in stop and len(word) > 2 and len(word.split())>1])
        if (rnumb % 100 == 1):
            print("processed %d reviews" % (rnumb))
        
print("****Done****")


processed 1 reviews
processed 101 reviews
processed 201 reviews
processed 301 reviews
processed 401 reviews
****Done****


In [117]:
k = 100
print("Using basic counter: ")
top_k_b_df =pandas.DataFrame([[key,value] for key,value in counter_basic.items()],columns=['Term','Frequency'])
top_k_b_df.sort_values('Frequency', axis=0, ascending=False).head(k)


Using basic counter: 


,Term,Frequency
1151,blank sleeve,15
1657,shopify store,14
1664,good luck,11
2171,[ prisync ],10
92,online store,10
1460,competitor prices,10
1707,shopify platform,9
2503,landing page,9
1175,shopify payments,7
90,ecommerce platform,6


In [118]:
print("Using Stem counter: ")
top_k_s_df =pandas.DataFrame([[key,value] for key,value in counter_stem.items()],columns=['Term','Frequency'])
top_k_s_df.sort_values('Frequency', axis=0, ascending=False).head(k)

Using Stem counter: 


,Term,Frequency
1518,shopify stor,17
2212,blank sleev,15
1176,online stor,14
1662,good luck,11
2165,[ prisync ],10
1636,competitor pric,10
1701,shopify platform,9
1354,landing pag,9
726,shopify pay,7
90,credit card,6


In [119]:
print("Using Lemmatizer counter: ")
top_k_l_df =pandas.DataFrame([[key,value] for key,value in counter_lemm.items()],columns=['Term','Frequency'])
top_k_l_df.sort_values('Frequency', axis=0, ascending=False).head(k)

Using Lemmatizer counter: 


,Term,Frequency
1151,blank sleeve,15
1657,shopify store,14
1664,good luck,11
2171,[ prisync ],10
92,online store,10
1460,competitor prices,10
1707,shopify platform,9
2503,landing page,9
1175,shopify payments,7
90,ecommerce platform,6


### MI and PMI Analysis



In [120]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas
import numpy as np
import math

from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize
from nltk.corpus import stopwords

In [121]:
! pip install --upgrade textblob

Requirement already up-to-date: textblob in ./common/.virtualenv/python3/lib/python3.4/site-packages
Requirement already up-to-date: nltk>=3.1 in /usr/local/lib/python3.4/dist-packages (from textblob)


In [122]:
from textblob import TextBlob


In [123]:
test = TextBlob("Shopify Capstone 2016 good")

In [124]:
test.sentiment.polarity

0.7

In [125]:
commentContent = commentKeepDf['bodyCopy'].as_matrix()

In [126]:
#commentKeepDf.shape

In [127]:
#Instantiation
sid = SentimentIntensityAnalyzer()

In [128]:
pdlist = []
##Assign Vader score to individual review using Vader compound score
##creating a list of the reviews along with their polarity score as assigned by Vader
##list containing a list where each element is [review,polarity]
for rownum, comment in enumerate(commentContent):
    ss = sid.polarity_scores(comment)
    textblobpol=TextBlob(comment).sentiment.polarity
    pdlist.append([comment,ss['compound'],textblobpol])
    if (rownum % 100 == 1):
            print("processed %d reviews" % (rownum+1))

            
print("****Done****")

processed 2 reviews
processed 102 reviews
processed 202 reviews
processed 302 reviews
processed 402 reviews
****Done****


In [129]:
test = sid.polarity_scores("Shopify Capstone 2016 good")
test['compound']


0.4404

In [130]:
commentDf = pandas.DataFrame(pdlist)
commentDf.columns = ['commentCol','vader','textBlob']
commentDf.head()

,commentCol,vader,textBlob
0,I would consider shopify. I know you said basi...,0.8990,0.350000
1,I had the same experience with a pre-order wit...,0.0000,0.000000
2,Looks like they're running Magento as their ca...,0.4215,0.099085
3,Thanks! We're definitely very lucky to be in t...,0.9912,0.119413
4,"A LOT of Shopify stories on here lately, or ma...",0.4588,-0.050000


In [131]:
print("Number of Positive Comments based on Vader: ",(commentDf.loc[commentDf['vader']>0]['vader']).size)
print("\nNumber of Positive Comments based on textBlob: ",(commentDf.loc[commentDf['textBlob']>0]['textBlob']).size)


Number of Positive Comments based on Vader:  367

Number of Positive Comments based on textBlob:  350


In [132]:
#for (index,word,wordCount) in top_k_l_df.itertuples():
    #print(index,word,wordCount)

In [133]:
#top_k_l_df

In [134]:
#Find out if a particular review has the word from topk list
#note: needed to ad .lower else not matching all words

top_k_l = counter_lemm.most_common(k)

freqReview = []
#create a list, one entry for each review; the entry is binary list indicating whether this review as the term i or not in the top k
for i in range(len(commentDf)):
    tempCounter = Counter([wnLemm.lemmatize(word.lower()) for word in extract_candidate_chunks(commentDf['commentCol'][i])])
    #extract_candidate_chunks(reviewDf['reviewCol'][i])
    #print(tempCounter)
    topkinComment = [1 if tempCounter[word] > 0 else 0 for (word,wordCount) in top_k_l]
    freqReview.append(topkinComment)

freqReviewDf = pandas.DataFrame(freqReview)
dfName = []
for c in top_k_l:
    dfName.append(c[0])
    #print(c)
freqReviewDf.columns = dfName
freqReviewDf.head()

,blank sleeve,shopify store,good luck,online store,competitor prices,[ prisync ],shopify platform,landing page,shopify payments,ecommerce platform,...,t-shirt shop,coupon codes,long way,personal experience,my answer,templating language,web applications,front end,indie seller,few logo designs and/or banners
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [135]:
finalcommentDf = commentDf.join(freqReviewDf)
#finalcommentDf.head()

In [136]:
finaldf = dataDf.join(finalcommentDf)
#finaldf.head()

#### Frequency Analysis of top k=100 terms in Positive and Negative comments (using Vader)

In [137]:
#top k positive comments

#Count the frequency of words
from collections import Counter
#https://pymotw.com/2/collections/counter.html
# counter is a container that keeps track of how many times equivalent values are added
import re
#A regular expression (or RE) specifies a set of strings that matches it
#A formal language for specifying strings
counter = Counter()
for rnumb,comment in enumerate(commentDf.loc[commentDf['vader']>0]['commentCol']):

        
        #add the word in its lowercase form to the counter Container if it is not a stop word and is greather than 2 characters
        counter.update([wnLemm.lemmatize(word.lower()) for word in extract_candidate_chunks(comment)  if word.lower() not in stop and len(word.split())>1])
        if (rnumb % 100 == 1):
            print("processed %d reviews" % (rnumb))
        

topkPos_v = counter.most_common(k)


print("\n \nTop k (100) from comments with positive vader scores: \n")
topkPos_v_df =pandas.DataFrame([[key,value] for key,value in counter.items()],columns=['Term','Frequency'])
topkPos_v_df.sort_values('Frequency', axis=0, ascending=False).head(k)


processed 1 reviews
processed 101 reviews
processed 201 reviews
processed 301 reviews

 
Top k (100) from comments with positive vader scores: 



,Term,Frequency
1022,blank sleeve,15
1482,shopify store,12
1487,good luck,11
1309,competitor prices,10
1947,[ prisync ],10
2238,landing page,9
80,online store,8
1762,little bit,6
1806,bank account,6
1527,shopify platform,6


In [138]:
#top k negative comments

#Count the frequency of words
from collections import Counter
#https://pymotw.com/2/collections/counter.html
# counter is a container that keeps track of how many times equivalent values are added
import re
#A regular expression (or RE) specifies a set of strings that matches it
#A formal language for specifying strings
counter = Counter()
for rnumb,comment in enumerate(commentDf.loc[commentDf['vader']<0]['commentCol']):

        
        #add the word in its lowercase form to the counter Container if it is not a stop word and is greather than 2 characters
        counter.update([wnLemm.lemmatize(word.lower()) for word in extract_candidate_chunks(comment)  if word.lower() not in stop and len(word.split())>1])
        if (rnumb % 100 == 1):
            print("processed %d reviews" % (rnumb))
        

topkNeg_v = counter.most_common(k)
print("\n \nTop k (100) from comments with negative vader scores: \n")
topkNeg_v_df =pandas.DataFrame([[key,value] for key,value in counter.items()],columns=['Term','Frequency'])
topkNeg_v_df.sort_values('Frequency', axis=0, ascending=False).head(k)

processed 1 reviews

 
Top k (100) from comments with negative vader scores: 



,Term,Frequency
167,custom fields,2
101,product line,2
48,same error,2
0,blog post,1
155,buyer name,1
156,templates cons,1
157,inventory line,1
158,footer design,1
159,label printer,1
160,newsletter sign,1


#### Frequency Analysis of top k=100 terms in Positive and Negative comments (using textBlob)

In [139]:

    #top k positive comments

#Count the frequency of words
from collections import Counter
#https://pymotw.com/2/collections/counter.html
# counter is a container that keeps track of how many times equivalent values are added
import re
#A regular expression (or RE) specifies a set of strings that matches it
#A formal language for specifying strings
counter = Counter()
for rnumb,comment in enumerate(commentDf.loc[commentDf['textBlob']>0]['commentCol']):

        
        #add the word in its lowercase form to the counter Container if it is not a stop word and is greather than 2 characters
        counter.update([wnLemm.lemmatize(word.lower()) for word in extract_candidate_chunks(comment)  if word.lower() not in stop and len(word.split())>1])
        if (rnumb % 100 == 1):
            print("processed %d reviews" % (rnumb))
        

topkPos_tB = counter.most_common(k)
print("\n \nTop k (100) from comments with positive textBlob scores: \n")
topkPos_tB_df =pandas.DataFrame([[key,value] for key,value in counter.items()],columns=['Term','Frequency'])
topkPos_tB_df.sort_values('Frequency', axis=0, ascending=False).head(k)


processed 1 reviews
processed 101 reviews
processed 201 reviews
processed 301 reviews

 
Top k (100) from comments with positive textBlob scores: 



,Term,Frequency
1458,good luck,11
1453,shopify store,10
1912,[ prisync ],10
1279,competitor prices,10
2185,landing page,9
81,online store,8
1725,little bit,6
1766,bank account,6
1020,shopify payments,6
388,self-service module,5


In [140]:
#top k negative comments

#Count the frequency of words
from collections import Counter
#https://pymotw.com/2/collections/counter.html
# counter is a container that keeps track of how many times equivalent values are added
import re
#A regular expression (or RE) specifies a set of strings that matches it
#A formal language for specifying strings
counter = Counter()
for rnumb,comment in enumerate(commentDf.loc[commentDf['textBlob']<0]['commentCol']):

        
        #add the word in its lowercase form to the counter Container if it is not a stop word and is greather than 2 characters
        counter.update([wnLemm.lemmatize(word.lower()) for word in extract_candidate_chunks(comment)  if word.lower() not in stop and len(word.split())>1])
        if (rnumb % 100 == 1):
            print("processed %d reviews" % (rnumb))
        

topkNeg_tB = counter.most_common(k)
print("\n \nTop k (100) from comments with negative textBlob scores: \n")
topkNeg_tB_df =pandas.DataFrame([[key,value] for key,value in counter.items()],columns=['Term','Frequency'])
topkNeg_tB_df.sort_values('Frequency', axis=0, ascending=False).head(k)


processed 1 reviews

 
Top k (100) from comments with negative textBlob scores: 



,Term,Frequency
102,blank sleeve,15
112,personal information,3
244,prepaid cards,3
10,** creep case,2
30,js framework,2
260,shopify store,2
88,false site,2
68,ecommerce platform,2
13,shopify platform,2
104,[ shopify ],2


### Mutual Information

#### Based on Vader

In [141]:
#creating a binary list indicating whether a review is positive (1) or negative (0)
gtScore = []
for i in range(len(finaldf)):
    if finaldf['vader'][i]>0:
        gtScore.append(1)
    else:
        gtScore.append(0)

In [142]:
#Calculate mutual information score using scikit lean package
import sklearn
import sklearn.metrics as metrics
miScore = []
for word in top_k_l:
    #two inputs for MI, both are binary, first indicate the class (positive or negative), second indicates if term is present in reveiw (0 = no , 1 = yes)
    #need to know which reviews are p or n and which actually contain the term to get MI
    miScore.append([word[0]]+[metrics.mutual_info_score(gtScore,finaldf[word[0]].as_matrix())])
    #question these are the MI scores for positive reviews, how do we get them for negative reviews
miScoredf = pandas.DataFrame(miScore).sort_values(1,ascending=0)
miScoredf.columns = ['Word','MI Score']
print("MI Scores based on Vader Sentiment: \n")
miScoredf.head(10)

MI Scores based on Vader Sentiment: 



,Word,MI Score
42,shopify app,0.416160
2,good luck,0.414505
82,same error,0.413387
89,custom fields,0.413387
10,little bit,0.413363
40,shopify plus,0.413264
14,self-service module,0.413136
20,new service model,0.413136
19,nike etc,0.413136
17,companies like mediamarkt,0.413136


#### Based on textBlob

In [143]:
#creating a binary list indicating whether a review is positive (1) or negative (0)
gtScore = []
for i in range(len(finaldf)):
    if finaldf['textBlob'][i]>0:
        gtScore.append(1)
    else:
        gtScore.append(0)
#Calculate mutual information score using scikit lean package
import sklearn
import sklearn.metrics as metrics
miScore = []
for word in top_k_l:
    #two inputs for MI, both are binary, first indicate the class (positive or negative), second indicates if term is present in reveiw (0 = no , 1 = yes)
    #need to know which reviews are p or n and which actually contain the term to get MI
    miScore.append([word[0]]+[metrics.mutual_info_score(gtScore,finaldf[word[0]].as_matrix())])
    #question these are the MI scores for positive reviews, how do we get them for negative reviews
miScoredf = pandas.DataFrame(miScore).sort_values(1,ascending=0)
miScoredf.columns = ['Word','MI Score']
print("MI Scores based on textBlob Sentiment: \n")

miScoredf.head(10)

MI Scores based on textBlob Sentiment: 



,Word,MI Score
42,shopify app,0.386479
57,personal information,0.386479
2,good luck,0.385723
31,prepaid cards,0.385231
12,use shopify,0.384800
10,little bit,0.384362
15,e-commerce companies,0.384092
4,competitor prices,0.384092
5,[ prisync ],0.384092
20,new service model,0.384092
